In [ ]:
# This is a retry of the voltage code but more efficient
epsilon_0=8.854187817* 10**(12) #This could have arbitrary units
math.pi

set_pH = 7
bound_pH = 1
lower_pH = set_pH-bound_pH
upper_pH = set_pH+bound_pH
radius = 9 #This is how far out the voltage goes before "decaying" to 0, make it odd so that the amino acid is in the center where V=0 by definition

protein_raw =  #This is the matrix that holds x,y,z,pI, assume it is in the (0,64)^3 octant of R^3, assume the matrix rows are in order of amino acid sequence
protein_data= np.append(protein_raw,numpy.zeros([len(a),5]),1) #This matrix holds x,y,z,pI,r,q,dx,dy,dz
protein_data[:,4] = np.sqrt(np.square(protein_data[:,0])+np.square(protein_data[:,1])+np.square(protein_data[:,2])) #This find the r for all amino acids

d_mat = np.zeros(protein_raw.shape(0),3) #This will hold the \vec d for the \vec p of dipole

aa_space = np.zeros(radius,radius,radius,1) #This is the space that will be used to calculate the voltage around amino acids without accessing the entire voltage matrix
voltage = np.zeros(64,64,64,2) #This matrix holds the scalar values of the electric potential at each point in space
distance_mat = np.zeros(radius,radius,radius,1) #This will hold the distances of each point from the "center" (radius/2,radius/2,radius/2) and also the x,y,z coordinates of that spot
r_mat = np.zeros(radius,radius,radius,3) #This matrix will hold the vector r in the space around the amino acid
center = np.array[radius/2,radius/2,radius/2] #The fake center to find distances in distance_mat

for i in range(distance_mat.shape(0)):
  for j in range(distance_mat.shape(1)):
    for k in range(distance_mat.shape(2)):
      distance_mat(i,j,k,0)=math.hypot(i-center(0),j-center(1),k-center(2)) #This fills up the distance_mat\

      r_mat(i,j,k,0)=i-center(0)
      r_mat(i,j,k,1)=j-center(1)
      r_mat(i,j,k,2)=k-center(2)

distance_mat(radius/2,radius/2,radius/2,0) = 1 #This avoid dividing by 0 later
distance_mat_rec = np.reciprocal(distance_mat.astype(float)) #This puts the distances in the denominator for later

pr_mat = np.zeros(radius,radius,radius,1) #\vec p \cdot \vec r matrix

for aa in range(protein_data.shape(0)-2): #Goes over each amino acid
  protein_data(aa+1,6) = protein_data(aa+2,0)-protein_data(aa,0) #The dx coordinate
  protein_data(aa+1,7) = protein_data(aa+2,1)-protein_data(aa,1) #The dy coordinate
  protein_data(aa+1,8) = protein_data(aa+2,2)-protein_data(aa,2) #The dz coordinate

  d_mat(aa,0) = protein_data(aa+2,0)-protein_data(aa,0) #The dx coordinate
  d_mat(aa,1) = protein_data(aa+2,1)-protein_data(aa,1) #The dy coordinate
  d_mat(aa,2) = protein_data(aa+2,2)-protein_data(aa,2) #The dz coordinate

#Now the algorithms to calculate the voltage
#Might need to pick a new voltage at the source of the monopole
for aa in range(protein_data.shape(0)): #Goes over each amino acid
  if protein_data(aa,3)>lower_pH || protein_data(i,3)<upper_pH: #If the amino acid is not charged

    protein_data(aa,5)=1

    #Dipole code

    pr_mat = protein_data(aa,5) * np.dot(d_mat,r_mat)

    aa_space = 1/(4*math.pi*epsilon_0) * np.multiply(np.square(distance_mat_rec),pr_mat)

    #Input this local space into the entire voltage space
    local_to_global(voltage,aa_space)

    #Reset aa_space
    aa_space = np.zeros(radius,radius,radius,1)

  elif protein_data(aa,3)<=lower_pH: #If the amino acid is negatively charged
    #Monopole code

    protein_data(aa,5)=-1
    aa_space=protein_data(aa,5)/(4*math.pi*epsilon_0) * distance_mat_rec #This is the voltage from a monopole

    #Input this local space into the entire voltage space
    local_to_global(voltage,aa_space)

    #Reset aa_space
    aa_space = np.zeros(radius,radius,radius,1)


  elif protein_data(aa,3)>=upper_pH: #If the amino acid is positively charged
    #Monopole code

    protein_data(aa,5)=1
    aa_space=protein_data(aa,5)/(4*math.pi*epsilon_0) * distance_mat_rec #This is the voltage from a monopole

    #Input this local space into the entire voltage space
    local_to_global(voltage,aa_space)

    #Reset aa_space
    aa_space = np.zeros(radius,radius,radius,1)


voltage

#Once we have the electric potential of the amino acid, we can find the energy of the system


In [ ]:
#This slices to put a matrix inside another
def local_to_global(voltage,aa_space):

  xb = voltage.shape[0]
  xa = aa_space.shape[0]
  yb = voltage.shape[1]
  ya = aa_space.shape[1]
  zb = voltage.shape[2]
  za = aa_space.shape[2]

  lower_x = (xb) // 2 - (xa // 2)
  upper_x = (xb // 2) + (xa // 2)
  lower_y = (yb) // 2 - (ya // 2)
  upper_y = (yb // 2) + (ya // 2)
  lower_z = (zb) // 2 - (za // 2)
  upper_z = (zb // 2) + (za // 2)
  voltage[lower_x:upper_x, lower_y:upper_y,lower_z:upper_z] = aa_space + voltage[lower_x:upper_x, lower_y:upper_y,lower_z:upper_z]